# GCP Billing Analysis Notebook

This notebook contains the billing data for further analysis.

## Data Overview
- **Date Range**: Last 14 days
- **Services**: Cloud Functions, Cloud Run, Firestore, etc.
- **Cost Tracking**: Daily breakdown by service

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots

# Set style
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")
pd.set_option("display.max_columns", None)

In [ ]:
# Load the data
df = pd.read_csv("daily_billing_data.csv")
daily_summary = pd.read_csv("daily_summary.csv")

# Convert date columns
df["date"] = pd.to_datetime(df["date"])
daily_summary["date"] = pd.to_datetime(daily_summary["date"])

print("Data loaded successfully!")
print(f'Date range: {df["date"].min()} to {df["date"].max()}')
print(f"Total records: {len(df)}")
print(f'Unique services: {df["service"].nunique()}')

In [ ]:
# Display data overview
print("\n=== Daily Summary ===")
print(daily_summary)

print("\n=== Service Cost Summary ===")
service_summary = df.groupby("service")["cost"].agg(["sum", "mean", "count"]).round(4)
service_summary.columns = ["Total Cost", "Average Daily Cost", "Days Tracked"]
print(service_summary.sort_values("Total Cost", ascending=False))

In [ ]:
# Create custom visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Daily cost trend
axes[0, 0].plot(daily_summary["date"], daily_summary["total_cost"], marker="o")
axes[0, 0].set_title("Daily Total Cost Trend")
axes[0, 0].set_xlabel("Date")
axes[0, 0].set_ylabel("Cost ($)")
axes[0, 0].tick_params(axis="x", rotation=45)

# 2. Service cost breakdown
service_totals = df.groupby("service")["cost"].sum().sort_values(ascending=True)
axes[0, 1].barh(service_totals.index, service_totals.values)
axes[0, 1].set_title("Total Cost by Service")
axes[0, 1].set_xlabel("Cost ($)")

# 3. Cost distribution
axes[1, 0].hist(daily_summary["total_cost"], bins=10, alpha=0.7)
axes[1, 0].set_title("Daily Cost Distribution")
axes[1, 0].set_xlabel("Cost ($)")
axes[1, 0].set_ylabel("Frequency")

# 4. Service count over time
service_counts = df.groupby("date")["service"].count()
axes[1, 1].plot(service_counts.index, service_counts.values, marker="s")
axes[1, 1].set_title("Number of Active Services")
axes[1, 1].set_xlabel("Date")
axes[1, 1].set_ylabel("Service Count")
axes[1, 1].tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Interactive Plotly dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Daily Cost Trend", "Service Breakdown", "Cost Distribution", "Service Count"),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]],
)

# Add traces
fig.add_trace(
    go.Scatter(x=daily_summary["date"], y=daily_summary["total_cost"],
              mode="lines+markers", name="Total Cost"),
    row=1, col=1,
)

for service in df["service"].unique():
    service_data = df[df["service"] == service]
    fig.add_trace(
        go.Bar(x=service_data["date"], y=service_data["cost"], name=service),
        row=1, col=2,
    )

fig.add_trace(
    go.Histogram(x=daily_summary["total_cost"], name="Cost Distribution"),
    row=2, col=1,
)

fig.add_trace(
    go.Scatter(x=service_counts.index, y=service_counts.values,
              mode="lines+markers", name="Service Count"),
    row=2, col=2,
)

fig.update_layout(height=800, title_text="GCP Billing Analysis Dashboard")
fig.show()

## Analysis Insights

### Key Findings:
1. **Cost Trends**: Analyze daily cost patterns
2. **Service Impact**: Identify which services contribute most to costs
3. **Optimization Opportunities**: Find areas for cost reduction
4. **Usage Patterns**: Understand service usage over time

### Next Steps:
- Set up automated billing alerts
- Implement cost optimization strategies
- Monitor usage patterns for anomalies
- Plan for scaling decisions